In [1]:
#importing the dependencies
import pandas as pd
import numpy as np
import time
import datetime
import os
from datetime import datetime as dt
from glob import glob

In [2]:
# Set display to max for rows and columns
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)

In [4]:
df = pd.read_csv(r'D:\Shyam\Documents\MUSER\Extraction\music-analysis.csv',encoding='latin-1')
df.head(5)

C:\Users\shyam\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,record_id,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_nano_time,event_seek_position_in_milliseconds,event_start_time,event_elapsed_time,song_album_id,song_album_name,song_artist_id,song_artist_name,song_bit_rate,song_date_added,song_disc_number,song_duration,song_file_size,song_file_format,song_id,song_last_played,song_name,song_file_path,song_play_count,song_playlist_id,playlist_play_order,podcast,song_sample_rate,track,song_release_year,album_id,album_name,album_artist_name,album_date_added,album_release_year,number_of_songs_in_album,number_of_discs_in_album,album_artists,album_paths,genre_id,genre_name,number_of_songs_for_genre,album_artist_name.1,album_artist_album_data,audio_devic_type,audio_volume_data,device_info_app_version,device_info_device_model,device_info_google_play_services_app,device_info_google_play_services_lib,device_info_ignoring_battery_optimizations,device_info_power_save_mode_enabled,device_info_sdk_version,device_info_sdk_version_int,device_info_talk_back_enabled,device_info_timestamp,event_current_time_in_iso_utc
0,0U3K5S2MPePNvsl6LMt1jJCq1nj1,1612558620337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3.0,Pixel,20.47.14 (100400-349456378),12451000.0,False,False,9.0,28.0,False,1.612559e+12,NaN
1,2AzUudC1tyQMklupZFYSU0pyPw43,1-28d1cc02-3472-4411-9bf7-2fc09c140051,PLAY,NaN,1.605021e+12,9.958191e+11,0.0,NaN,NaN,1.0,Download,1.0,Download,320 kbps,1.565624e+09,0.0,32392.0,1.2 MB,MPEG-1 Layer 3,53.0,0.0,new Fortnite Scenario Emote Music,/storage/emulated/0/Download/new Fortnite Scen...,0.0,0.0,0.0,False,44.1 kHz,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-10T15:13:30.887Z
2,2AzUudC1tyQMklupZFYSU0pyPw43,2-094667b7-3480-4b82-a479-1c9e630a8e86,PAUSE,NaN,1.605021e+12,9.978555e+11,1887.0,NaN,NaN,1.0,Download,1.0,Download,320 kbps,1.565624e+09,0.0,32392.0,1.2 MB,MPEG-1 Layer 3,53.0,0.0,new Fortnite Scenario Emote Music,/storage/emulated/0/Download/new Fortnite Scen...,0.0,0.0,0.0,False,44.1 kHz,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-10T15:13:32.923Z
3,2AzUudC1tyQMklupZFYSU0pyPw43,6-d13c3a9f-f47e-4543-a1b9-dfc87f514253,PLAY,NaN,1.605022e+12,1.795041e+12,0.0,NaN,NaN,1.0,Download,1.0,Download,320 kbps,1.565624e+09,0.0,32392.0,1.2 MB,MPEG-1 Layer 3,53.0,0.0,new Fortnite Scenario Emote Music,/storage/emulated/0/Download/new Fortnite Scen...,0.0,0.0,0.0,False,44.1 kHz,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-10T15:26:50.109Z
4,2AzUudC1tyQMklupZFYSU0pyPw43,7-9bdc57f2-1d2f-44a3-9861-10d3d7f06045,PAUSE,NaN,1.605022e+12,1.796623e+12,1345.0,NaN,NaN,1.0,Download,1.0,Download,320 kbps,1.565624e+09,0.0,32392.0,1.2 MB,MPEG-1 Layer 3,53.0,0.0,new Fortnite Scenario Emote Music,/storage/emulated/0/Download/new Fortnite Scen...,0.0,0.0,0.0,False,44.1 kHz,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-10T15:26:51.691Z


In [5]:
#sorting the current timestamp in ascending order
df.sort_values('event_current_time_in_milliseconds', ascending=True, inplace=True, ignore_index = True)

In [6]:
# Removing rows that were used for internal testing on user participant devices
df = df.drop(df.loc[26323:26353].index) # userid = 68737007
df = df.drop(df.loc[26380:26455].index) # userid = 68737004 that has two additional different muser user ids 
df = df.drop(df.loc[25660:25665].index) # userid = 68737003 
df = df.drop(df.loc[26354:26379].index) # userid = 68737003

In [7]:
# changing date type from utc iso to different date type
df['event_current_time_other'] = pd.to_datetime(df['event_current_time_in_iso_utc'])

In [8]:
#adding a new column event_current_time_ms_to_date to the csv 
df['event_current_time_ms_to_date'] = pd.to_datetime(df['event_current_time_in_milliseconds'], unit='ms')

In [9]:
#creating column- elapsed_time_in_min 
df['elapsed_time_in_min'] = round(df['event_seek_position_in_milliseconds']/60000,4)
#df.dtypes

In [10]:
#converting song_last_played to current timestamp
df['song_last_played_date'] = pd.to_datetime(df['song_last_played'], unit='ms')

In [11]:
#remove the empty valued audio_volume_data column
df = df.dropna(axis=0, subset=['audio_volume_data'])
df.shape

(23344, 61)

In [12]:
#function to extract minimum volume from an array of audio_volume_data
def pull_vol_min(i):
    a,b,c,d= i.split(',')
    return c

In [13]:
df['min_volume'] =df['audio_volume_data'].apply(pull_vol_min)
df['min_volume'] = df['min_volume'].str.split('=').str[1]

In [14]:
#function to extract current volume from an array of audio_volume_data
def pull_vol_curr(i):
    a,b,c,d,e= i.split()
    start = b.find('[')
    end = b.find(',')
    substring = b[start+1:end]
    return substring

In [15]:
# apply the function and split the column by '='
df['curent_volume'] = df.audio_volume_data.apply(pull_vol_curr)
df['curent_volume'] = df['curent_volume'].str.split('=').str[1]

In [16]:
#filtering out the records that are greater than the minimum volume value
df= df[ df['curent_volume'] > df['min_volume'] ]

In [17]:
# Create new variable curr_time to change the current time into milliseconds
curr_time = pd.to_datetime(df['event_current_time_in_iso_utc'])

# Create new column curr_time_in_ms
df['curr_time_in_ms'] = pd.to_datetime(curr_time, unit='ms')

In [18]:
# convert the new column to display milliseconds
df['curr_time_in_ms'] = df['curr_time_in_ms'].astype(np.int64) / int(1e6)

In [19]:
#creating column- current time in minutes 
df['curr_time_in_min'] = round(df['curr_time_in_ms']/60000,4)

In [20]:
#reset the indexes of the dataset
df.reset_index(inplace=True)

In [21]:
# shorten the final file by only keeping required columns in the final csv file
user_behavior_data = df[["user_id", "record_id", "song_id", "song_name", "player_event_type", "ui_event_type",  "event_current_time_in_milliseconds", "event_seek_position_in_milliseconds", "song_album_name", "song_artist_name", "song_duration", "song_name", "audio_volume_data","event_current_time_ms_to_date", "event_current_time_in_iso_utc", "event_current_time_other", "min_volume", "curent_volume", "elapsed_time_in_min","song_last_played_date", "song_play_count"]]

In [26]:
user_behavior_data.head(25)

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,event_current_time_in_iso_utc,event_current_time_other,min_volume,curent_volume,elapsed_time_in_min,song_last_played_date,song_play_count,listening_time
0,MkwuKLaebsQaNrbviX0dul0nCSH2,54-cd93a8fb-8b04-4d03-b4f9-3f7cc7d0bb2b,578.0,The Great Pretender,NaN,PLAY,1.587251e+12,83350.0,Forever Gold,"Platters, The",162247.0,The Great Pretender,"VolumeData [currentVolumeLevel=1, volumeMax=15...",2020-04-18 23:05:43.422000128,2020-04-18T23:05:43.422Z,2020-04-18 23:05:43.422000+00:00,-2147483648,1,1.3892,1970-01-01 00:00:00.000,1.0,NaN
1,MkwuKLaebsQaNrbviX0dul0nCSH2,55-f3197e2f-ca9a-4603-ba04-1b1d65dde7b4,578.0,The Great Pretender,PLAY,NaN,1.587251e+12,0.0,Forever Gold,"Platters, The",162247.0,The Great Pretender,"VolumeData [currentVolumeLevel=1, volumeMax=15...",2020-04-18 23:05:43.512999936,2020-04-18T23:05:43.513Z,2020-04-18 23:05:43.513000+00:00,-2147483648,1,0.0000,1970-01-01 00:00:00.000,1.0,NaN
2,MkwuKLaebsQaNrbviX0dul0nCSH2,56-ef751b72-f51f-464c-b072-5b307a801288,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=2, volumeMax=15...",2020-04-18 23:05:46.316000000,2020-04-18T23:05:46.316Z,2020-04-18 23:05:46.316000+00:00,-2147483648,2,0.0000,NaT,NaN,NaN
3,MkwuKLaebsQaNrbviX0dul0nCSH2,57-97bf10d2-a1da-4ecb-8f31-60fefd7ec3d6,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=3, volumeMax=15...",2020-04-18 23:05:47.307000064,2020-04-18T23:05:47.307Z,2020-04-18 23:05:47.307000+00:00,-2147483648,3,0.0000,NaT,NaN,NaN
4,MkwuKLaebsQaNrbviX0dul0nCSH2,58-dbc9f9bd-40c8-4ec3-86fe-9884cd48e433,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=4, volumeMax=15...",2020-04-18 23:05:47.676000000,2020-04-18T23:05:47.676Z,2020-04-18 23:05:47.676000+00:00,-2147483648,4,0.0000,NaT,NaN,NaN
5,MkwuKLaebsQaNrbviX0dul0nCSH2,59-c3e25a15-68d7-44c8-a26a-89077eb5463c,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=5, volumeMax=15...",2020-04-18 23:05:48.064999936,2020-04-18T23:05:48.065Z,2020-04-18 23:05:48.065000+00:00,-2147483648,5,0.0000,NaT,NaN,NaN
6,MkwuKLaebsQaNrbviX0dul0nCSH2,60-bdac7424-a470-4e96-a850-11355337c180,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=6, volumeMax=15...",2020-04-18 23:05:48.664999936,2020-04-18T23:05:48.665Z,2020-04-18 23:05:48.665000+00:00,-2147483648,6,0.0000,NaT,NaN,NaN
7,MkwuKLaebsQaNrbviX0dul0nCSH2,61-8843f5e3-7c7e-4537-9c32-40045b7532f8,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=7, volumeMax=15...",2020-04-18 23:05:49.440000000,2020-04-18T23:05:49.440Z,2020-04-18 23:05:49.440000+00:00,-2147483648,7,0.0000,NaT,NaN,NaN
8,MkwuKLaebsQaNrbviX0dul0nCSH2,62-2e1a0322-451c-4200-a3ce-fd4b3de82d5b,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2020-04-18 23:05:50.135000064,2020-04-18T23:05:50.135Z,2020-04-18 23:05:50.135000+00:00,-2147483648,8,0.0000,NaT,NaN,NaN
9,MkwuKLaebsQaNrbviX0dul0nCSH2,63-5d7abb68-1458-486a-ba45-f74f456ceafe,NaN,NaN,NaN,VOLUME_UP,1.587251e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=9, volumeMax=15...",2020-04-18 23:05:50.752999936,2020-04-18T23:05:50.753Z,2020-04-18 23:05:50.753000+00:00,-2147483648,9,0.0000,NaT,NaN,NaN


In [27]:
# LISTENING TIME

# Original Code
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  listening_time = 0
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY": #this is covering same song pause-play
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):#when idx-1 is play and idx is either play or next or prev
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


KeyboardInterrupt: 

In [ ]:
# USER EVENT

# Duration for half song duration
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  user_half_song = 0  
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if ui_event == "PLAY" and user_behavior_data.loc[idx, "ui_event_type"] == "PLAY":
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            user_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "user_half_song"] = user_half_song/(1000*60)
    if (ui_event == "PAUSE" or ui_event == "NEXT" or ui_event == "PREV") and (user_behavior_data.loc[idx-1, "ui_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "ui_event_type"] == "PLAY"):
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            user_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "user_half_song"] = user_half_song/(1000*60)

In [28]:
# PLAYER EVENT TYPE

# Duration for half song duration
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  ui_event = user_behavior_data.loc[idx, "ui_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  song_duration_time = user_behavior_data.loc[idx, "song_duration"]
  event_seek = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  player_half_song = 0  
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY":
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            player_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "player_half_song"] = player_half_song/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):
        if user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"] >= user_behavior_data.loc[idx, "song_duration"]/2:
            player_half_song = user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
            user_behavior_data.loc[idx, "player_half_song"] = player_half_song/(1000*60)

C:\Users\shyam\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


In [ ]:
# TIME SPENT IN BETWEEN PAUSE AND PLAY EVENTS

# CODE IN PROGRESS

In [29]:
#path = r'A:\MUSER\data\cleaned_data\mergedFiles\dummy\analysis_Code'
path = r'D:\Shyam\Documents\MUSER\Analysis\User Cleaned Data'

user_behavior_data.to_csv(os.path.join(path,'muser_master.csv'), index=False, encoding='utf-8-sig')